#<center> U-net Implimentation

## Importing required libraries

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D, Conv2DTranspose
from keras.models import Model

In [ ]:
# function to build u-net model
def build_model(image_shape, start_neurons=64, classes=1, activation='sigmoid',):
    # Contraction Path
    input_layer = Input((image_shape))

    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)
    
    # Expansion Path
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    
    output_layer = Conv2D(filters=classes, kernel_size=(3,3), padding="same", use_bias=True, activation=activation)(uconv1)

    model = Model(inputs=[input_layer], outputs=[output_layer],  name="U-Net")
    
    return model


model = build_model(image_shape = (256,256,3), start_neurons = 64)

In [ ]:
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_143 (Conv2D)             (None, 256, 256, 64) 1792        input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_144 (Conv2D)             (None, 256, 256, 64) 36928       conv2d_143[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_32 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_144[0][0]                 
______________________________________________________________________________________________

### Calculating trainable parameter:

**number of learnable param  = (kernel size x kernel size x number of output feature channel x previous number of feature channel) + number of output feature channel**

Example:

Layer (type) | Output Shape | Param #
---|---|---
input_9 (InputLayer) | (None, 256, 256, 3) | 0
conv2d_143 (Conv2D) | (None, 256, 256, 64) | 1792
conv2d_144 (Conv2D) |(None, 256, 256, 64)  | 36928

<br />

From above table

input layer has 0 learnable parameter.

conv2d_143 = (3*3*64*3) + 64 = 1792

conv2d_144 = (3*3*64*64) + 64 = 36928

<br />

### Output shape of image when applied convolution:

**You can use this formula [(W−K+2P)/S]+1**

* W is the input volume 
* K is the Kernel size
* P is the padding
* S is the stride

Example:

convolution applied to image of size 128X128X3 and 40 filters of size 5X5 

Output_Shape = (128-5+0)/1 + 1 = 124

Output_Shape = (124,124,40)

40 is the number of filters
